In [1]:
import requests
from requests.exceptions import ConnectionError
import lxml.html as lh
import pandas as pd
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
companies1 = pd.read_csv('List_of_companies.csv', encoding = "ISO-8859-1")
companies = pd.read_csv('List_of_companies.csv', encoding = "ISO-8859-1")

print(companies.dtypes)

companies['Security'] = companies['Security'].str.replace(u'\xa0', u' ')#this was messing with the classes
companies['Security'] = companies['Security'].str.replace(r' \(Class A\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class B\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r' \(Class C\)', '')#some companies are liste multiple times because of class of stock
companies['Security'] = companies['Security'].str.replace(r'\(', '')#need to remove parenthesis too for the url
companies['Security'] = companies['Security'].str.replace(r'\)', '')
companies['Security'] = companies['Security'].str.replace(r'\.com', '-com')#for amazon and salesforce
companies['Security'] = companies['Security'].str.replace(r'Corporation', 'corp')#corporation got shortened to corp
companies['Security'] = companies['Security'].str.replace(r'corporation', 'corp')
companies['Security'] = companies['Security'].str.replace(r'Company', 'co')#company got shortened to co
companies['Security'] = companies['Security'].str.replace(r'company', 'co')
companies['Security'] = companies['Security'].str.replace(r'&', 'and')#& is and for some reason
companies['Security'] = companies['Security'].str.replace(r' ', '-')#the url uses hyphens a lot
companies['Security'] = companies['Security'].str.replace(r'.', '')#don't want these here
companies['Security'] = companies['Security'].str.replace(r',', '')
companies['Security'] = companies['Security'].str.replace(r'!', '')
companies['Security'] = companies['Security'].str.replace(r'\'', '')

companies['Security'] = companies['Security'].str.lower()#lowercases everything
print(len(companies))
companies

Symbol                   object
Security                 object
SEC filings              object
GICS Sector              object
GICS Sub-Industry        object
Headquarters Location    object
Date first added         object
CIK                       int64
Founded                  object
dtype: object
505


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3m-co,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902
1,ABT,abbott-laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
2,ABBV,abbvie-inc,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
3,ABMD,abiomed-inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
4,ACN,accenture-plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989
5,ATVI,activision-blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",8/31/2015,718877,2008
6,ADBE,adobe-inc,reports,Information Technology,Application Software,"San Jose, California",5/5/1997,796343,1982
7,AMD,advanced-micro-devices-inc,reports,Information Technology,Semiconductors,"Santa Clara, California",3/20/2017,2488,1969
8,AAP,advance-auto-parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",7/9/2015,1158449,1932
9,AES,aes-corp,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",10/2/1998,874761,1981


In [3]:
#getting url
for n in range(10):
    currName = companies.at[n, 'Security']
    url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
    print(url)


https://www1.salary.com/3m-co-Executive-Salaries.html
https://www1.salary.com/abbott-laboratories-Executive-Salaries.html
https://www1.salary.com/abbvie-inc-Executive-Salaries.html
https://www1.salary.com/abiomed-inc-Executive-Salaries.html
https://www1.salary.com/accenture-plc-Executive-Salaries.html
https://www1.salary.com/activision-blizzard-Executive-Salaries.html
https://www1.salary.com/adobe-inc-Executive-Salaries.html
https://www1.salary.com/advanced-micro-devices-inc-Executive-Salaries.html
https://www1.salary.com/advance-auto-parts-Executive-Salaries.html
https://www1.salary.com/aes-corp-Executive-Salaries.html


In [4]:
#Makes a list of to make the graph later on
col=[('Company Name', []), ('CEO Name', []), ('CEO Pay', []), ('Median Employee Pay', []), ('CEO Pay Ratio', [])]

for n in range(len(companies)):
    #getting url
    currName = companies.at[n, 'Security']
    currName1 =companies1.at[n, 'Security']
    col[0][1].append(currName1)
    url='https://www1.salary.com/' + currName + '-Executive-Salaries.html'
    page=requests.get(url)
    
    skip = False
    #getting all the tables
    try:
        doc=lh.fromstring(page.content)
        tr_elements=doc.xpath('//tr')
    except:
        skip = True
    
    #indicates that we found the magics on this particular page
    done = True
    
    if skip:
        print(currName1)
        print(currName)
        print(url)
        col[1][1].append('')
        col[2][1].append(0)
        col[3][1].append(0)
        col[4][1].append(0)
    else:
        #this will find the table that has the relevant info for the CEO,and put the correct info into col
        for j in range(0,len(tr_elements)):
            #T is our j'th row
            T=tr_elements[j]

            #Only check rows of size 4 (with CEO Name, CEO Pay, Median Employee Pay,
            #and CEO Pay Ratio)
            if len(T)==4:
                #checks if the current row has the 4 maagic words
                table_Name = ['','','','']
                i=0
                for t in T:
                    name = t.text_content()
                    name = name.__str__()
                    table_Name[i]=name
                    i+=1

                #if it does, I'll take the data from the row right below it
                if (table_Name[0] == 'CEO Name' and table_Name[1] == 'CEO Pay' and table_Name[2] == 'Median Employee Pay' and table_Name[3] == 'CEO Pay Ratio'):
                    #setting done to false
                    done = False
                    T=tr_elements[j+1]
                    #i is the index of our column
                    i=0

                    #this is a temporary storage of the content of each line of the csv
                    content = []

                    #Iterate through each element of the row
                    for t in T.iterchildren():
                        #this part if for cleaning the cell contents
                        data=t.text_content() #gets the content of cell
                        data = data.__str__() #turns content to string
                        data = data.replace(r'$', '') #takes out dollar sign for number conversion
                        data = data.replace(r',', '') #takes out comma for number conversion
                        data = data.strip() #clears our \r and \n and spaces, cleans up the content string
                        data = data.replace(r'CEO Pay', '') #cleans up the content string
                        data = data.replace(r'Median Employee Pay', '') #cleans up the content string
                        data = data.replace(r'CEO Pay Ratio', '') #cleans up the content string
                        data = data.replace(r'Ratio', '') #cleans up the content string
                        data = data.replace(r':1', '') #takes out :1 in the ratio for number conversion
                        data = data.strip() #clears our \r and \n and spaces again to finish cleaning up

                        content.append(data)

                        #Check if row is empty
                        if i>0:
                        #Convert any numerical value to integers
                            try:
                                data=int(data)
                            except:
                                pass
                        #Append the data to col
                        col[i + 1][1].append(data)
                        #Increment i for the next column
                        i+=1
                    break
        if done:
            print(currName1)
            print(currName)
            print(url)
            col[1][1].append('')
            col[2][1].append(0)
            col[3][1].append(0)
            col[4][1].append(0)

Abbott Laboratories
abbott-laboratories
https://www1.salary.com/abbott-laboratories-Executive-Salaries.html
Accenture plc
accenture-plc
https://www1.salary.com/accenture-plc-Executive-Salaries.html
Activision Blizzard
activision-blizzard
https://www1.salary.com/activision-blizzard-Executive-Salaries.html
Advance Auto Parts
advance-auto-parts
https://www1.salary.com/advance-auto-parts-Executive-Salaries.html
AES Corp
aes-corp
https://www1.salary.com/aes-corp-Executive-Salaries.html
Alexandria Real Estate Equities
alexandria-real-estate-equities
https://www1.salary.com/alexandria-real-estate-equities-Executive-Salaries.html
Alexion Pharmaceuticals
alexion-pharmaceuticals
https://www1.salary.com/alexion-pharmaceuticals-Executive-Salaries.html
Align Technology
align-technology
https://www1.salary.com/align-technology-Executive-Salaries.html
Allegion
allegion
https://www1.salary.com/allegion-Executive-Salaries.html


KeyboardInterrupt: 

In [ ]:
#making the dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [ ]:
#displaying the dataframe
df

In [ ]:
int = 0
for i in range(len(companies)):
    if (df.at[i, 'Median Employee Pay'] == 0):
        int += 1
int

In [ ]:
int = 0
for i in range(len(companies)):
    if ('(' in df.at[i, 'Company Name']):
        int += 1
        print(i)
int

In [ ]:
df.at[22, 'Company Name']

In [ ]:
companies.at[22, 'Security']

In [ ]:
companies1.at[22, 'Security']